# Построим baseline

## Импорт библиотек

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from preprocess import CustomPreprocessor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, mean_absolute_error,r2_score)
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import pandas as pd


## Считывание данных

In [14]:
df = pd.read_csv('vehicles.csv')

In [15]:
df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)

## Удалим ошибочные данные

In [16]:
df_train = df_train[(df_train['price'] <= 500_000) & (df_train['price'] >= 100)]
df_train = df_train[df_train['odometer'] <= 600_000]

df_test = df_test[(df_test['price'] <= 500_000) & (df_test['price'] >= 100)]
df_test = df_test[df_test['odometer'] <= 600_000]

## Удаляем из тренировочных данных выбросы и строки с большим количеством пропусков

In [17]:
def clean_data(df: pd.DataFrame,train:bool) -> pd.DataFrame:
    if train:
        # Удаляем строки с большим количеством пропусков
        df = df.dropna(thresh=int(df.shape[1]*0.7))
        
        # Удаление выбросов по IQR для числовых колонок
        num_cols = df.select_dtypes(exclude='object').columns.tolist()
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            if IQR > 0:
                low = Q1 - 1.5*IQR
                high = Q3 + 1.5*IQR
                df = df[(df[col] >= low) & (df[col] <= high)]
    # удалим колонки с большим кол-во уник знач
    df.drop(columns=['id', 'url', 'image_url', 'region_url','VIN','county','description','posting_date','model'],inplace=True)
    return df


In [18]:
df_train_clean = clean_data(df_train,True)
df_test_clean = clean_data(df_test,False)


num_cols = df_train_clean.select_dtypes(exclude='object').columns.tolist()
cat_cols = df_train_clean.select_dtypes(include='object').columns.tolist()

for col in num_cols:
    if df_train_clean[col].isna().all():
        df_train_clean[col] = 0
        df_test_clean[col] = 0
    else:
        median_value = df_train_clean[col].median()
        df_train_clean[col].fillna(median_value, inplace=True)
        df_test_clean[col].fillna(median_value, inplace=True)

for col in cat_cols:
    if df_train_clean[col].isna().all():
        df_train_clean[col] = "missing"
        df_test_clean[col] = "missing"
    else:
        mode_value = df_train_clean[col].mode()[0]
        df_train_clean[col].fillna(mode_value, inplace=True)
        df_test_clean[col].fillna(mode_value, inplace=True)


/var/folders/ym/hx_pjxld1rn9q46zklb197b80000gn/T/ipykernel_74609/3925096447.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_clean[col].fillna(median_value, inplace=True)
/var/folders/ym/hx_pjxld1rn9q46zklb197b80000gn/T/ipykernel_74609/3925096447.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

## Сравним несколько моделей и выберем наилучшую 

### Подготовим данные для обучения,сделав разбиение на train и test 

In [19]:
TARGET_COL = "price"

X_train = df_train_clean.drop(columns=["price"])
X_valid = df_test_clean.drop(columns=["price"])
y_train = df_train_clean['price']
y_valid = df_test_clean['price']

## Для начало рассмотрим стандартную Линейную регрессию 

### Для начала закодируем наши категориальные признаки и отмаштабируем их(необходимо для работы с линейными моделями)

In [20]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_features = X_train.select_dtypes(exclude=["object"]).columns
cat_features = X_train.select_dtypes(include=["object"]).columns

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
])

X_train_ready = preprocessor.fit_transform(X_train)
X_valid_ready = preprocessor.transform(X_valid)

In [21]:
model_regres = LinearRegression()
model_regres.fit(X_train_ready,y_train)
y_pred = model_regres.predict(X_valid_ready)

print('Метрики LinearRegression:')
print('MAE: ',mean_absolute_error(y_valid,y_pred))
print('RMSE: ',np.sqrt(mean_squared_error(y_valid,y_pred)))
print('r2: ',r2_score(y_valid,y_pred))


Метрики LinearRegression:
MAE:  6899.358328102942
RMSE:  12177.702786330286
r2:  0.3632694732058199


## Далее рассмотрим DissisonTree и RandomForest

### Закодируем категориальные признаки

In [22]:
X_train_tree = X_train.copy()
X_valid_tree = X_valid.copy()

cat_features = X_train_tree.select_dtypes(include=["object", "category"]).columns

label_encoders = {}

for col in cat_features:
    le = LabelEncoder()
    
    combined = pd.concat([X_train_tree[col], X_valid_tree[col]], axis=0).astype(str)
    le.fit(combined)
    
    X_train_tree[col] = le.transform(X_train_tree[col].astype(str))
    X_valid_tree[col] = le.transform(X_valid_tree[col].astype(str))
    
    label_encoders[col] = le

print(f"Label Encoding завершён. Закодировано признаков: {len(cat_features)}")

Label Encoding завершён. Закодировано признаков: 12


In [23]:
X_train_tree.shape

(280139, 16)

In [24]:
# DecisionTree
model_tree = DecisionTreeRegressor()
model_tree.fit(X_train_tree, y_train)
y_pred_tree = model_tree.predict(X_valid_tree)

print('Метрики DecisionTreeRegressor:')
print('MAE: ', mean_absolute_error(y_valid, y_pred_tree))
print('RMSE: ', np.sqrt(mean_squared_error(y_valid, y_pred_tree)))
print('R2: ', r2_score(y_valid, y_pred_tree))

# RandomForest

model_rf = RandomForestRegressor(
    n_estimators=100,       # количество деревьев, можно 50 для быстрого теста
    max_depth=16,           # ограничение глубины, чтобы избежать переобучения
    min_samples_leaf=3,     # минимальное число объектов в листе
    max_features='sqrt',    # sqrt(num_features) на каждом разбиении
    n_jobs=-1,              # использовать все ядра CPU
    random_state=42,
    verbose=1               # вывод прогресса
)

# Обучение
model_rf.fit(X_train_tree, y_train)

# Предсказание
y_pred_rf = model_rf.predict(X_valid_tree)

# Метрикчя
print('Метрики RandomForestRegressor:')
print('MAE: ', mean_absolute_error(y_valid, y_pred_rf))
print('RMSE: ', np.sqrt(mean_squared_error(y_valid, y_pred_rf)))
print('R2: ', r2_score(y_valid, y_pred_rf))

Метрики DecisionTreeRegressor:
MAE:  3745.4543003681056
RMSE:  9459.61171452306
R2:  0.6157874452302391


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s


Метрики RandomForestRegressor:
MAE:  4474.295798476559
RMSE:  8696.856388309378
R2:  0.6752497243436929


[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


### Рассмотрим градиентные бустинги

### Для начала обучим catboost

In [25]:
from catboost import CatBoostRegressor

cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

cat_boost_model = CatBoostRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='MAE',
    eval_metric='MAE',
    random_seed=42,
    verbose=200,
    early_stopping_rounds=100
    )

cat_boost_model.fit(
    X_train,
    y_train,
    cat_features=cat_features,
    eval_set=(X_valid, y_valid),
    use_best_model=True,
)

y_pred = cat_boost_model.predict(X_valid)

mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
r2 = r2_score(y_valid, y_pred)

print("Метрики CatBoost:")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

0:	learn: 10082.1150483	test: 11050.0612633	best: 11050.0612633 (0)	total: 218ms	remaining: 3m 38s
200:	learn: 4101.9338186	test: 4951.1193076	best: 4951.1193076 (200)	total: 1m 5s	remaining: 4m 19s
400:	learn: 3839.7178045	test: 4681.3089039	best: 4681.3089039 (400)	total: 2m 10s	remaining: 3m 14s
600:	learn: 3711.8337889	test: 4560.1637745	best: 4560.1637745 (600)	total: 3m 6s	remaining: 2m 3s
800:	learn: 3615.9651405	test: 4467.9703966	best: 4467.9703966 (800)	total: 3m 52s	remaining: 57.8s
999:	learn: 3540.2148667	test: 4395.3219799	best: 4395.3219799 (999)	total: 4m 40s	remaining: 0us

bestTest = 4395.32198
bestIteration = 999

Метрики CatBoost:
MAE: 4395.321980861931
RMSE: 8721.288155474902
R2: 0.6734225427894758


### Обучим ligthgbm

In [26]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Категориальные признаки как category
X_train_lgb = X_train.copy()
X_valid_lgb = X_valid.copy()
cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

for col in cat_features:
    X_train_lgb[col] = X_train_lgb[col].astype('category')
    X_valid_lgb[col] = X_valid_lgb[col].astype('category')

lgb_model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=-1, 
    random_state=42,
    verbose=-1,
    early_stopping_rounds=50  
)

lgb_model.fit(
    X_train_lgb,
    y_train,
    eval_set=[(X_valid_lgb, y_valid)],
    eval_metric='mae',
    categorical_feature=cat_features 
)

y_pred = lgb_model.predict(X_valid_lgb)

mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
r2 = r2_score(y_valid, y_pred)

print("Метрики LightGBM:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

Метрики LightGBM:
MAE: 4094.9166
RMSE: 8019.7817
R2: 0.7238


### Далее рассмотрим xgboost

In [27]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

X_train_xgb = X_train.copy()
X_valid_xgb = X_valid.copy()

label_encoders = {}
cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([X_train[col], X_valid[col]], axis=0)
    le.fit(combined)
    X_train_xgb[col] = le.transform(X_train_xgb[col])
    X_valid_xgb[col] = le.transform(X_valid_xgb[col])
    label_encoders[col] = le

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    verbosity=0,  
    early_stopping_rounds=50,
    eval_metric='mae'
)

xgb_model.fit(
    X_train_xgb,
    y_train,
    eval_set=[(X_valid_xgb, y_valid)],
    verbose=False 
)

y_pred_xgb = xgb_model.predict(X_valid_xgb)

mae_xgb = mean_absolute_error(y_valid, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_valid, y_pred_xgb))
r2_xgb = r2_score(y_valid, y_pred_xgb)

print("Метрики XGBoost:")
print(f"MAE: {mae_xgb:.4f}")
print(f"RMSE: {rmse_xgb:.4f}")
print(f"R2: {r2_xgb:.4f}")

Метрики XGBoost:
MAE: 4166.3989
RMSE: 8184.9142
R2: 0.7124
